# Chatbot

In this notebook you will begin to create chatbot functionality, creating an AI bot capable of retaining conversation history.

## Helper Functions

### Amazon SageMaker
Use Amazon SageMaker Jumpstart to deploy a Llama2-70B model. 

In [1]:
%pip install --upgrade --quiet sagemaker

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


***
You can continue with the default model or choose a different model: this notebook will run with the following model IDs :
- `meta-textgeneration-llama-2-7b-f`
- `meta-textgeneration-llama-2-13b-f`
- `meta-textgeneration-llama-2-70b-f`
***

In [2]:
model_id, model_version = "meta-textgeneration-llama-2-70b-f", "2.*"

## Deploy model

***
You can now deploy the model using SageMaker JumpStart.
***

In [3]:
from sagemaker.jumpstart.model import JumpStartModel

model = JumpStartModel(model_id=model_id,
                       model_version=model_version)
predictor = model.deploy()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/codespace/.config/sagemaker/config.yaml


For forward compatibility, pin to model_version='2.*' in your JumpStartModel or JumpStartEstimator definitions. Note that major version upgrades may have different EULA acceptance terms and input/output signatures.
Using model 'meta-textgeneration-llama-2-70b-f' with wildcard version identifier '2.*'. You can pin to version '2.0.4' for more stable results. Note that models may have different input/output signatures after a major version upgrade.
Couldn't call 'get_role' to get Role ARN from role name upgradhenryisaac to get Role path.


ClientError: An error occurred (ValidationException) when calling the CreateModel operation: 1 validation error detected: Value 'arn:aws:iam::114997964775:user/upgradhenryisaac' at 'executionRoleArn' failed to satisfy constraint: Member must satisfy regular expression pattern: ^arn:aws[a-z\-]*:iam::\d{12}:role/?[a-zA-Z_0-9+=,.@\-_/]+$

In [5]:
endpoint_name = predictor.endpoint_name

import boto3, json
sm_client = boto3.client('sagemaker-runtime')

def transform_input(prompt: str, temperature) -> bytes:
    input_str = json.dumps({
        "inputs": [[{"role": "user", "content": prompt},]],
        "parameters": {"max_new_tokens": 512,"top_p": 0.9,"temperature": temperature}
    })
    return input_str.encode('utf-8')
    
def transform_output(output: bytes) -> str:
    response_json = json.loads(output['Body'].read().decode("utf-8"))
    return response_json[0]["generation"]["content"]

def generate(prompt, temperature = 0.6):
    response = sm_client.invoke_endpoint(
        EndpointName=endpoint_name, 
        CustomAttributes="accept_eula=true", 
        ContentType="application/json",
        Body=transform_input(prompt, temperature)
    )
    response_output = transform_output(response)
    return response_output

### Edit System Message

In [6]:
def prompt_with_system_message(prompt, system_message):
    prompt = f"""
    <s>[INST] <<SYS>>{system_message}<</SYS>>

    User: {prompt}
    Agent:[/INST]
    """
    return prompt

### Include One-to-many Shot Learning

In [7]:
def prompt_with_examples(prompt, system_message, examples=[]):
    
    # Start with the initial part of the prompt with system message
    full_prompt = f"<s>[INST] <<SYS>>{system_message}<</SYS>>\n"

    # Add each example to the prompt
    for user_msg, agent_response in examples:
        full_prompt += f"{user_msg} [/INST] {agent_response} </s><s>[INST]"

    # Add the main prompt and close the template
    full_prompt += f"{prompt} [/INST]"

    return full_prompt

### LlamaChatbot Class

In [8]:
class LlamaChatbot:
    def __init__(self, system_message):
        self.system_message = system_message
        self.conversation_history = []  # list of tuples (user_msg, agent_response)

    def chat(self, user_msg):
        # Generate the prompt using the conversation history and the new user message
        prompt = prompt_with_examples(user_msg, self.system_message, self.conversation_history)
        
        # Get the model's response
        agent_response = generate(prompt)

        # Store this interaction in the conversation history
        self.conversation_history.append((user_msg, agent_response))

        return agent_response

    def reset(self):
        # Clear conversation history
        self.conversation_history = []

## No Conversation Memory

In [9]:
prompt = "Hello my name is Rock. Nice to meet you!"

print(generate(prompt))

 Hello Rock! It's great to meet you too. Is there anything you'd like to chat about or ask? I'm here to help with any questions you might have.


In [10]:
prompt = "Can you remind me what my name is?"

print(generate(prompt))

 I'm not able to access personal information, including your name. However, I can suggest ways for you to remind yourself of your name.

1. Check your identification documents: Your name should be listed on your driver's license, passport, or other government-issued ID cards.
2. Look at your social media profiles: Your name should be listed on your social media profiles, such as Facebook, Twitter, or LinkedIn.
3. Check your email address: Your name may be included in your email address, especially if you use a personalized email address that includes your name.
4. Ask a friend or family member: If you're having trouble remembering your name, you can ask a friend or family member to remind you.
5. Check your personal belongings: Your name may be listed on your personal belongings, such as your phone, laptop, or other devices.

I hope these suggestions help you remind yourself of your name! If you continue to have trouble remembering your name, you may want to consider speaking with a he

In [11]:
system_message = """
You are a friendly chatbot always eager to help and engage in meaningful conversation. \
You always remember the details of our previous conversations, \
especially if a user gives them their name.
"""

prompt = "Hello my name is Rock. Nice to meet you!"

print(generate(prompt_with_system_message(prompt, system_message)))

 Hello Rock! It's great to meet you too! I'm just an AI, I'm here to help answer any questions you have or just chat with you if you'd like. It's nice to see you here!

By the way, Rock, I'm glad you introduced yourself. I'll be sure to remember your name for next time we chat. Is there anything you'd like to talk about or ask me? I'm all ears!


In [12]:
system_message = """
You are a friendly chatbot always eager to help and engage in meaningful conversation. \
You always remember the details of our previous conversations, \
especially if a user gives them their name.
"""

prompt = "Can you remind me what my name is?"

print(generate(prompt_with_system_message(prompt, system_message)))

 Sure thing! Your name is [User's Name]. It's great to see you again! How can I assist you today? Is there anything new you'd like to chat about or ask for help with?


## Create Conversation Memory

In [13]:
class LlamaChatbot:
    def __init__(self, system_message):
        self.system_message = system_message
        self.conversation_history = []  # list of tuples (user_msg, agent_response)

    def chat(self, user_msg):
        # Generate the prompt using the conversation history and the new user message
        prompt = prompt_with_examples(user_msg, self.system_message, self.conversation_history)
        
        # Get the model's response
        agent_response = generate(prompt)

        # Store this interaction in the conversation history
        self.conversation_history.append((user_msg, agent_response))

        return agent_response

    def reset(self):
        # Clear conversation history
        self.conversation_history = []

In [14]:
system_message = """
You are a friendly chatbot always eager to help and engage in meaningful conversation.
"""

chatbot = LlamaChatbot(system_message)

In [15]:
print(chatbot.chat("Hi, my name is Rock. Nice to meet you!"))

 Hello Rock! It's great to meet you too! I'm just an AI, my name is ChatGenie, and I'm here to help you with any questions or topics you'd like to discuss. How's your day going so far? Is there anything you'd like to chat about or ask me? I'm all ears! 😊


In [16]:
print(chatbot.chat("Can you remind me what my name is?"))

 Of course, Rock! Your name is Rock. 😊 How's your day going so far? Is there anything else you'd like to chat about or ask me? I'm here to help!


In [17]:
chatbot.reset()

In [18]:
print(chatbot.chat("Can you remind me what my name is?"))

 Of course, I'd be happy to help! Unfortunately, I don't have access to personal information, so I'm unable to remind you of your name. However, I'd love to chat with you and help with any other questions you might have! Is there anything else I can assist you with? 😊
